In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
total_len = len(train)
average_values = train.mean(numeric_only=True)
train = train.fillna(average_values)
print(f'training exemples : {len(train)},  {(1-len(train)/total_len)*100:.1f} % data not NaN')
train.head()

In [ ]:
corr_mat = train.corr()
plt.figure(figsize=(20,20), dpi=100)
sns.heatmap(corr_mat,annot=False, fmt=".2f")
plt.show()

In [ ]:
sns.distplot(train["loss"])

In [ ]:
X = train.drop(["id","loss"],axis=1)
y = train["loss"]

In [ ]:
!pip install flaml

In [ ]:
from flaml import AutoML
automl = AutoML()

In [ ]:
automl.fit(X, y,task="regression",metric='rmse',time_budget=60*5*1)# 5mins

[](http://)

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best rmse on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
print(f'len : {len(test)}')
test.head()

In [ ]:
total_len = len(test)
average_values = test.mean(numeric_only=True)
test = test.fillna(average_values)
print(f'training exemples : {len(test)},  {(1-len(test)/total_len)*100:.1f} % data not NaN')
test.head()

In [ ]:
from sklearn.metrics import mean_absolute_error
import pickle
# 保存模型
with open("./automl_v2.pkl", "wb") as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)
 
# 加载模型并预测
with open("./automl_v2.pkl", "rb") as f:
    automl = pickle.load(f)
    
X_test = test.drop(["id"],axis=1)
pred = automl.predict(X_test)

test["loss"] = pred
test.head()

In [ ]:
test.index.name = 'id'

In [ ]:
submission = pd.DataFrame(
    {'id':test["id"] ,
     'loss': test["loss"]},columns=['id', 'loss'])

In [ ]:
submission.to_csv("submission.csv", index=False)
submission.head(10)